### Load in a MODFLOW-only version of the Sagehen model

In [ ]:
import os
import sys
from math import*
import matplotlib.pyplot as plt
import numpy as np

import config

# run installed version of flopy or add local path
try:
    import flopy
except:
    fpth = os.path.abspath(os.path.join('..', '..'))
    sys.path.append(fpth)
    import flopy

print(sys.version)
print('flopy version: {}'.format(flopy.__version__))

In [ ]:
nam_file = "uzftest1.nam"
model_ws = os.path.join("UZF_testproblem1")
mf = flopy.modflow.Modflow.load(nam_file, model_ws=model_ws, check=False, exe_name=config.mfnwtexe, verbose=True)

In [ ]:
mf.upw.sy = 0.2
mf.change_model_ws(new_pth='sagetemp', reset_external=False)

In [ ]:
lmt = flopy.modflow.ModflowLmt(mf, output_file_name='therm.ftl', output_file_header='extended',
                               output_file_format='formatted', package_flows=['all'])

In [ ]:
mf.write_input()
mf.run_model()

In [ ]:
mt = flopy.mt3d.Mt3dms(modflowmodel=mf, modelname='Therm', model_ws=mf.model_ws,
                       version='mt3d-usgs', namefile_ext='mtnam', exe_name=config.mt3dusgsexe,
                       ftlfilename='Therm.ftl', ftlfree=True)

In [ ]:
print(mf.upw.sy[0,20:30,30:40])

#### Governing equation for thermal transport
\begin{equation*}
\left( 1+ \frac{1- \theta}{\theta} \frac{\rho_s}{\rho} \frac{c_{P_{solid}}}{c_{P_{fluid}}} \right) \frac{\partial \left( \theta T \right) }{\partial t}
= \nabla \cdot \left[ \theta \left( \frac {k_{T_{bulk}}}{\theta \rho c_{Pfluid}} + \alpha \frac{\mathrm{q}}{\theta} \right) \cdot \nabla T \right] - \nabla \cdot \left( \mathrm{ \mathbf{q}} T \right) - q'_s T_s
\end{equation*}

#### Some constants we will be using

\begin{equation*}
k_{T_{fluid}} = 0.58 \frac {\mathrm{W}}{ \left( m \cdot K \right)}
\end{equation*}

\begin{equation*}
k_{T_{solid}} = 3.0 \frac {\mathrm{W}}{ \left( m \cdot K \right)}
\end{equation*}

\begin{equation*}
c_{P_{fluid}} = 4183 \frac{\mathrm{J}}{\frac{\mathrm{kg}}{\mathrm{K}}}
\end{equation*}

\begin{equation*}
c_{P_{solid}} = 710 \frac{\mathrm{J}}{\frac{\mathrm{kg}}{\mathrm{K}}}  
\end{equation*} 

\begin{equation*}
k_{T_{bulk}} = \theta k_{T_{fluid}} + \left( 1- \theta \right) k_{T_{solid}}
\end{equation*}

\begin{equation*}
\rho_{solid} = 2650 \frac{kg}{m^3}
\end{equation*}

#### Note

\begin{equation*}
\rho_b = \rho_s \left( 1- \theta \right)
\end{equation*}

\begin{equation*}
R = 1 + \frac{\rho_b}{\theta} k_{d_{temp}}
\end{equation*}

#### Thermal distribution factor to account for exchange with matrix and retardation of fluid heat flow
\begin{equation*}
k_{d_{temp}} = \frac {c_{P_{solid}}}{\rho c_{P_{fluid}}}
\end{equation*}

#### Bulk thermal diffusivity coefficient to account for bulk heat conduction in matrix and fluid
\begin{equation*}
D_{m_{Temp}} = \frac {k_{T_{bulk}}}{\theta \rho c_{P_{fluid}}}
\end{equation*}




Calculate these values for use in the MT3D-USGS input files

In [ ]:
Ktf = 0.58
Kts = 3.0
theta = 0.20

Ktb = theta * Ktf + (1-theta)*Kts
print("Ktb = " + str(Ktb) + " W/m/K")

In [ ]:
rho = 1000.    # kg/m^3
cps = 710.
cpf = 4183.

Kdt = cps / (rho * cpf)
print("Kdt = " + str(Kdt) + " m3/kg")

In [ ]:
rhos = 2650
rhob = rhos * (1-theta)

In [ ]:
# Ktb calculated above

Dmt = Ktb / (theta * rho * cpf)
print("Dmt = " + str(Dmt) + " m^2/s = " + str(Dmt*86400) + " m^2/day")
Dmt = Dmt * 86400

In [ ]:
# Have a look at the temperature forcing input
sample = mf.dis.nper
x = np.arange(sample)
y = 10 * np.sin((x/140 - 70) + 1)  #=(SIN(A3/90 - 70)+1)*10
plt.plot(x, y)
plt.xlabel('sample(n)')
plt.ylabel('voltage(V)')
plt.show()

In [ ]:
ncomp   =   1
lunit   =   'FT'
sconc   =   1.0
prsity  =   0.2    # mf.upw.sy[0,:,:]
cinact  =  -1.0
thkmin  =   0.00001
nprs    =  -1
nprobs  =  10
nprmas  =  10
dt0     =   1.0
nstp    =   1
mxstrn  = 500
ttsmult =   1.0
ttsmax  = 100

btn = flopy.mt3d.Mt3dBtn(mt, MFStyleArr=True, DRYCell=True, lunit=lunit,
                         sconc=sconc, ncomp=ncomp, prsity=prsity, cinact=cinact,
                         thkmin=thkmin, nprs=nprs, nprobs=nprobs, 
                         chkmas=True,nprmas=nprmas, dt0=dt0, nstp=nstp, 
                         mxstrn=mxstrn, ttsmult=ttsmult, ttsmax=ttsmax)

# ADV package
mixelm =    0
percel =    1.0000
mxpart = 5000
nadvfd =    1      # (1 = Upstream weighting)

adv = flopy.mt3d.Mt3dAdv(mt, mixelm=mixelm, percel=percel, mxpart=mxpart, 
                         nadvfd=nadvfd)

# GCG solver package
mxiter =   1
iter1  = 200
isolve =   3
ncrs   =   0
accl   =   1.000000
cclose =   1.00e-06
iprgcg =   5

gcg = flopy.mt3d.Mt3dGcg(mt, mxiter=mxiter, iter1=iter1, 
                         isolve=isolve, ncrs=ncrs, accl=accl, 
                         cclose=cclose, iprgcg=iprgcg)

# SSM
mxss = 100
ssm = flopy.mt3d.Mt3dSsm(mt, mxss=mxss)

# DSP
al     =  0.5        # longitudinal dispersivity
trpt   =  0.1        # ratio of the horizontal transverse dispersivity to 'AL'
trpv   =  0.1        # ratio of the vertical transverse dispersitvity to 'AL'
dmcoef =  Dmt        # calculated above

dsp = flopy.mt3d.Mt3dDsp(mt, al=al, trpt=trpt, trpv=trpv, dmcoef=dmcoef,
                         multiDiff=True)

# UZT (Input temperature boundary)

# set up a 3D array of input temperatures
cuzinf  = np.ones((mf.dis.nper, mf.dis.nrow, mf.dis.ncol))
for i in range(y.shape[0]):
    cuzinf[i,:,:] *= y[i]

mxuzcon = np.count_nonzero(mf.uzf.iuzfbnd.array)
iet     = 1
iuzfbnd = mf.uzf.iuzfbnd.array
wc      = 0.115
sdh     = mf.dis.botm[0,:,:]
cuzinf  = {i: cuzinf[i] for i in range(cuzinf.shape[0])}

uzt = flopy.mt3d.Mt3dUzt(mt, mxuzcon=mxuzcon, iet=iet, iuzfbnd=iuzfbnd, wc=wc, sdh=sdh, 
                         cuzinf=cuzinf, cuzet=0, cgwet=0, filenames='Therm.uzt')


nsfinit = abs(mf.sfr.nstrm)
mxsfbc  = abs(mf.sfr.nstrm)
dispsf  = np.ones((abs(mf.sfr.nstrm)))
coldsf  = np.ones((abs(mf.sfr.nstrm)))
obs_sf  = [nsfinit - 1]
gage_output = [None, None, 'Therm.sftobs']


sft = flopy.mt3d.Mt3dSft(mt, nsfinit=nsfinit, mxsfbc=mxsfbc, icbcsf=81, ioutobs=82,
                         isfsolv=1, cclosesf=1.0E-5, mxitersf=50, crntsf=1.0, iprtxmd=0,
                         coldsf=coldsf, dispsf=dispsf, nobssf=1, obs_sf=obs_sf,  # sf_stress_period_data = sf_stress_period_data,
                         filenames=gage_output)


In [ ]:
isothm    = 1  # 1: Linear; 2: Freundlich; 3: Langmuir; 4: 1st-order kinetic sorption
ireact    = 0  # 0: no kinetic rate reaction; 1: 1st-order irreversible reaction
irctop    = 2  # >=2: rct variables entered as arrays; <2: rct variables 1D by layer
igetsc    = 0
rhob      = rhob  # calculated above
srconc    = np.zeros((mf.dis.nlay, mf.dis.nrow, mf.dis.ncol))

sp1       = Kdt      # Distribution Coefficient (Kd) (Calculated above)
sp2       = 0.0      # read, but not used with isothm = 1

rct = flopy.mt3d.Mt3dRct(mt, isothm=isothm, ireact=ireact, igetsc=igetsc,
                         sp1=sp1, sp2=sp2, srconc=srconc)

In [ ]:
mt.write_input()

# NOTE: Need to take out MXUZCON manually
mt.run_model()


In [ ]:
mf.upw.sy[0,9,49]

In [ ]:
mf.upw.sy = 0.2